In [1]:
%env TOKENIZERS_PARALLELISM=false
!pip install bertopic

env: TOKENIZERS_PARALLELISM=false
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 56.4 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=819467 sha256=71c388dec1cf7bc58cfdef35ca4ab65255c500152152722c2b37b134a7d50e8f
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=b5d6f5192160ecb9445f713e86b8b681d410d17e8809fa5fd74f0017a46e89d6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/

In [2]:
import pandas as pd
# we are doing word stuff so we don't really need any of the numerical columns
usecols=['text', 'speaker', 'season', 'episode', 'scene', 'utterance'][:2]
df = pd.read_csv(filepath_or_buffer='/kaggle/input/friends/friends.csv', usecols=usecols)
# we are going to throw away very short documents
df['token count'] = df['text'].apply(lambda x: len(x.split()))
df = df[df['token count'] > 2].drop(columns=['token count'])
df.head()

,text,speaker
0,There's nothing to tell! He's just some guy I ...,Monica Geller
1,"C'mon, you're going out with the guy! There's ...",Joey Tribbiani
2,"All right Joey, be nice. So does he have a hum...",Chandler Bing
3,"Wait, does he eat chalk?",Phoebe Buffay
4,"(They all stare, bemused.)",Scene Directions


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54751 entries, 0 to 67371
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     54751 non-null  object
 1   speaker  54672 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [4]:
# let's build our stopword collection
# we collected these from the wordcloud package in a previous run
wordcloud_stopwords = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'com', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'else', 'ever', 'few', 'for', 'from', 'further', 'get', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'hence', 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'however', 'http', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', 'just', 'k', "let's", 'like', 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'otherwise', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'r', 'same', 'shall', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'since', 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', "there's", 'therefore', 'these', 'they', "they'd", "they'll", "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where', "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't", 'www', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']
# let's add in some stopwords from looking at topic -1 in prior runs
prior_stopwords = ['Oh', 'Im', 'dont', 'know', 'No', 'What', 'great', 'believe', 'Well', 'thing', 'um', 'feel', 'way', 'uh', 'youre', 'You', 'Youre',
                  'thats', 'theres', 'right', 'think', 'Yeah', 'We', 'said', 'Ooh', 'things', 'little', 'gonna', 'want',
                   'And', 'one', 'got', 'The', 'really', 'So', 'All', 'Its', 'go', 'see', 'Okay', 'Thats', 'need',  'guys', 'Go',  'back', 'Ill', 'wanna', 'mean', 'cant', 'play', 'Hey', 'going',
                   'getting', 'Scene', 'look', 'now', 'tell', 'good', 'us', 'take', 'time', 'well', 'something', 'Hi', 'Look', 'didnt',
                   'Do', 'lot', 'better', 'new', 'guy', 'much', 'big', 'This', 'give', 'maybe', 'But', 'That', 'make', 'Yknow', 'sorry', 'say', 'Just', 'help',
                   'Ive', 'talking', 'will', 'II', 'come', 'Wow', 'never', 'yknow', 'okay', 'wait', 'night', 'nothing', 'together', 'actually', 'hundred', 'yeah', 'man', 'enough',
                  'uhoh', 'Uhoh', 'OK', 'iswhat', 'Are', 'knowJust', ]
stopwords = sorted(wordcloud_stopwords + prior_stopwords)
print(stopwords[:20])

['All', 'And', 'Are', 'But', 'Do', 'Go', 'Hey', 'Hi', 'II', 'Ill', 'Im', 'Its', 'Ive', 'Just', 'Look', 'No', 'OK', 'Oh', 'Okay', 'Ooh']


In [5]:
from arrow import now
from bertopic import BERTopic
from plotly.express import histogram
from sklearn.feature_extraction.text import CountVectorizer

# we can dial this up and down for testing; a full run takes nearly seven minutes
SAMPLE_FRACTION = 0.99
time_start = now()
large_model = BERTopic(verbose=True, top_n_words=20, nr_topics=None, language='english', 
                       vectorizer_model=CountVectorizer(stop_words=stopwords, lowercase=False))
large_topics, large_probs = large_model.fit_transform(df['text'].sample(frac=SAMPLE_FRACTION, random_state=2023).dropna().values)
print('After {} our topic says we have {} topics.'.format(now() - time_start, len(set(large_topics))))
print('And we have {} documents with no topic ({}%).'.format(large_topics.count(-1), round(100 * large_topics.count(-1)/len(large_topics))))
histogram(x=[item for item in large_topics if item > -1], title = 'topic > -1 histogram')

/opt/conda/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

Batches:   0%|          | 0/1694 [00:00<?, ?it/s]

2023-10-20 16:21:03,696 - BERTopic - Transformed documents to Embeddings
2023-10-20 16:22:01,944 - BERTopic - Reduced dimensionality
2023-10-20 16:22:05,320 - BERTopic - Clustered reduced embeddings


After 0:05:13.536782 our topic says we have 771 topics.
And we have 17368 documents with no topic (32%).


In [6]:
from plotly.express import scatter
def visualize(arg_model, search_term):
    words = []
    values = []
    topic_result = arg_model.get_topic(arg_model.find_topics(search_term=search_term, top_n=1)[0][0],)
    for item in topic_result:
        if item[0] not in {search_term, search_term + 's'}: # drop self and possessive/plural
            words.append(item[0])
            values.append(item[1])
    scatter(x=words, y=values, title=search_term).show()

for term in ['Phoebe', 'Rachel', 'Chandler', 'relationship', 'breakup', 'Marcel', 'U2', 'leather', 'armadillo']:
    visualize(arg_model=large_model, search_term=term)